In [1]:
from sklearn.model_selection import train_test_split
from transformers import BertModel, BertConfig, BertTokenizer, AutoTokenizer
import datasets
from datasets import Dataset
import pandas as pd
import tensorflow as tf
import numpy as np
import nltk
import nltk.translate.meteor_score as meteor
from nltk import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
import keras
#import wandb
import torch
from torch.utils.data import TensorDataset, DataLoader
import pickle
from transformers import BigBirdPegasusForConditionalGeneration, TFTrainer, TFTrainingArguments, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, Trainer, TrainingArguments, BartForConditionalGeneration, DataCollatorForSeq2Seq

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/ubuntu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [3]:
torch.cuda
torch.cuda.is_available()

True

In [4]:
# cuda issues with this model
#path = "/home/ubuntu/Documents/OliverVinzelberg/led_large_16384_pubmed_CTS_untouched_blind_bestsofar_3072_1024_2bs_20e"

# this model works
path = "/home/ubuntu/Documents/OliverVinzelberg/led_large_16384_pubmed_CTS_untouched_3072_1024_2bs_20e"
model = AutoModelForSeq2SeqLM.from_pretrained(path, local_files_only=True)
model.cuda()

tokenizer = AutoTokenizer.from_pretrained(path, local_files_only=True)

In [5]:
#max_input_length = 3072
max_input_length = 4096
max_output_length = 1024

def longformer_preprocess_func_cts(batch):
    # tokenize the inputs and labels
    inputs = tokenizer(
        batch["text"],
        padding="max_length",
        truncation=True,
        max_length=max_input_length)
    
    outputs = tokenizer(
        batch["summary"],
        padding="max_length",
        truncation=True,
        max_length=max_output_length)

    batch["input_ids"] = inputs.input_ids
    batch["attention_mask"] = inputs.attention_mask

    # create 0 global_attention_mask lists
    batch["global_attention_mask"] = len(batch["input_ids"]) * [
        [0 for _ in range(len(batch["input_ids"][0]))]
    ]

    # since above lists are references, the following line changes the 0 index for all samples
    batch["global_attention_mask"][0][0] = 1
    batch["labels"] = outputs.input_ids

    # We have to make sure that the PAD token is ignored
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in labels]
        for labels in batch["labels"]
    ]

    return batch

In [6]:
def generate_sum(data):
    to_pred = tokenizer(data, padding="max_length", max_length=4096, return_tensors="pt", truncation=True)
    #to_pred = tokenizer(data, padding="max_length", max_length=3072, return_tensors="pt", truncation=True)
    input_ids=to_pred["input_ids"].cuda()
    attention_mask=to_pred["attention_mask"].cuda()
    #global attention on special tokens
    global_attention_mask = torch.zeros_like(attention_mask)
    global_attention_mask[:, 0] = 1
    
    predicted_ids = model.generate(input_ids, attention_mask=attention_mask, global_attention_mask=global_attention_mask)
    
    return tokenizer.batch_decode(predicted_ids, skip_special_tokens=True)

In [7]:
with open('/home/ubuntu/Documents/OliverVinzelberg/pickle_files/CTS_data_processed_removed_backslashN.pkl', 'rb') as handle:
    df_cts = pickle.load(handle)
    
cts_dataset = Dataset.from_pandas(df_cts[["text", "summary"]])

In [8]:
print(cts_dataset['text'][0])

EU Clinical Trials Register Clinical trial results: A Phase 2, 2-Stage, 2-Cohort Study of Talazoparib (BMN 673) Administered to Germline BRCA Mutation Subjects with Locally Advanced and/or Metastatic Breast Cancer Summary EudraCT number 2013-003076-12 Trial protocol GB ES Global end of trial date 31 October 2018 Results information Result version number v2 (current) This version publication date 07 September 2019 First version publication date 17 September 2017 Version creation reason Trial information Trial identification Sponsor protocol code 673-201 Additional study identifiers ISRCTN number - ClinicalTrials.gov id (NCT number) NCT02034916 WHO universal trial number (UTN) - Other trial identifiers Alias Study Number: C3441008 Notes: Sponsors Sponsor organisation name Pfizer Inc. Sponsor organisation address 235 E 42nd Street, New York, United States, NY 10017 Public contact Pfizer ClinicalTrials.gov Call Center, Pfizer Inc., 001 8007181021, ClinicalTrials.gov_Inquiries@pfizer.com Sc

In [14]:
raw_text = cts_dataset['text'][0]
max_input = 4096

chunked_input = [raw_text[i:i+max_input] for i in range(0, len(raw_text), max_input)]

print(len(chunked_input[13]))

192


In [15]:
for i in range(0, len(chunked_input)):
    output[i] = generate_sum(chunked_input[i])

In [24]:
out_str = ''
for i in output:
    out_str += ' '+i[0]
    
print(out_str)

 WHY WAS THIS STUDY DONE? What was the purpose of this study? Breast cancer is the name for cancer that starts in the breast. Some women with breast cancer have breast cancer that has spread to other parts of the body, such as the brain, spinal cord, or spinal cord. This type of cancer is called metastatic breast cancer.  Talazoparib is a medicine that is being tested for the treatment of breast cancer in women. It is used to treat patients with advanced or metastatic cancer of the breast, and it has been shown that it may be effective in treating patients with metastatic or locally advanced breast cancer (metastatic).  TalAZOPARIB (BMN 673) Is a medicine approved for use in patients with certain types of breast cancers.  WHAT HAPPENED DURING THE STUDY?  This study had 2 parts: a “Leading-inhibitor” part of the study and an “investigator-blinded” stage. A “lead-in-patient” is a person who was randomly assigned to 1 of 2 treatment groups by chance alone.  Lead-inpatient was a woman who 

In [8]:
print(cts_dataset['summary'][0])

 WHY WAS THIS STUDY DONE? Breast cancer is one of the most common causes of cancer in women. In 2012, around 1.7 million women were diagnosed with this disease around the world. Men can also get breast cancer, though this is quite rare.  Some breast cancers are caused by “mutations” in “DNA” in the BReast CAncer “genes” 1 and 2, which are more commonly known as BRCA1 and BRCA2. DNA is a material inside the cells of all living things that controls how cells in the body grow.  Genes are parts of DNA that contain the code for a specific feature, like eye color.  Mutations are changes in the DNA that can alter how a cell will behave.  If the BRCA1 and BRCA2 genes have mutations then this may cause breast cancer.  These mutations can be inherited from a parent with this mutation or through chance, which is like the roll of a dice. Special proteins known as “poly (ADP-ribose) polymerase” or “PARP” are found in all cells, and these proteins can help damaged cells fix or repair themselves so t

In [16]:
import pickle

with open('processed_tokenized_cts_dataset.pkl', 'wb') as handle:
    pickle.dump(processed_tokenized_cts_dataset, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [15]:
import pickle

with open('processed_tokenized_cts_dataset.pkl', 'rb') as handle:
    processed_tokenized_cts_dataset = pickle.load(handle)
